<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-Packages" data-toc-modified-id="Loading-Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading Packages</a></span></li><li><span><a href="#Prepping-files-for-SynChro---Finding-Orthologous-pairs" data-toc-modified-id="Prepping-files-for-SynChro---Finding-Orthologous-pairs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepping files for SynChro - Finding Orthologous pairs</a></span><ul class="toc-item"><li><span><a href="#C.-reinhardtii" data-toc-modified-id="C.-reinhardtii-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>C. reinhardtii</a></span></li><li><span><a href="#C.-incerta" data-toc-modified-id="C.-incerta-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>C. incerta</a></span></li><li><span><a href="#C.schloesseri" data-toc-modified-id="C.schloesseri-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>C.schloesseri</a></span></li><li><span><a href="#JG5-1" data-toc-modified-id="JG5-1-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>JG5-1</a></span></li><li><span><a href="#C.-malaysian" data-toc-modified-id="C.-malaysian-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>C. malaysian</a></span></li></ul></li><li><span><a href="#BLASTx---Finding-Orthologous-pairs" data-toc-modified-id="BLASTx---Finding-Orthologous-pairs-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>BLASTx - Finding Orthologous pairs</a></span><ul class="toc-item"><li><span><a href="#Function---Modify-GFF-files" data-toc-modified-id="Function---Modify-GFF-files-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Function - Modify GFF files</a></span></li><li><span><a href="#Applying-GFF-function" data-toc-modified-id="Applying-GFF-function-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Applying GFF function</a></span></li><li><span><a href="#Getting-longest-transcript/protein" data-toc-modified-id="Getting-longest-transcript/protein-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Getting longest transcript/protein</a></span></li><li><span><a href="#Running-BLASTx" data-toc-modified-id="Running-BLASTx-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Running BLASTx</a></span></li></ul></li><li><span><a href="#Rob's-Code" data-toc-modified-id="Rob's-Code-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Rob's Code</a></span><ul class="toc-item"><li><span><a href="#BLAST-line-class" data-toc-modified-id="BLAST-line-class-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>BLAST line class</a></span></li><li><span><a href="#Assess-the-similarity-of-pairs" data-toc-modified-id="Assess-the-similarity-of-pairs-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Assess the similarity of pairs</a></span></li><li><span><a href="#Parse-Reciprocal-Best-BLAST-Hits-(RBH)-into-pairs" data-toc-modified-id="Parse-Reciprocal-Best-BLAST-Hits-(RBH)-into-pairs-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Parse Reciprocal Best BLAST Hits (RBH) into pairs</a></span></li><li><span><a href="#Combining-RBH-Orthologous-Pairs" data-toc-modified-id="Combining-RBH-Orthologous-Pairs-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Combining RBH Orthologous Pairs</a></span></li><li><span><a href="#Combining-Filtered-RBH-Orthologous-Pairs" data-toc-modified-id="Combining-Filtered-RBH-Orthologous-Pairs-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Combining Filtered RBH Orthologous Pairs</a></span></li><li><span><a href="#Sequence-Divergence-of-Orthologous-pairs" data-toc-modified-id="Sequence-Divergence-of-Orthologous-pairs-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Sequence Divergence of Orthologous pairs</a></span></li></ul></li><li><span><a href="#Finding-Mating-Type-using-BLASTn" data-toc-modified-id="Finding-Mating-Type-using-BLASTn-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Finding Mating Type using BLASTn</a></span></li></ul></div>

# Loading Packages

In [1]:
import pandas as pd
import os, re
from itertools import combinations
%load_ext rpy2.ipython

# Prepping files for SynChro - Finding Orthologous pairs

## C. reinhardtii

Please note that in the following code:
- Mitochondrion DNA = Chromosome 18 
- Chloroplast DNA = Chromosome 19
- Mating type locus = Chromosome 20

In [ ]:
names = ['chr', 'prediction', 'type', 'start', 'end', 'extra1', 'sens', 'extra2', 'extra3']
chlam_def = pd.read_csv('/research/references/chlamydomonas/6.1_chlamy/annotation/CreinhardtiiCC_4532_707_v6.1.gene.gff3', delimiter = '\t', skiprows = 6, names = names)
chlam_def['name'] = chlam_def['extra3'].str.split(';', expand = True)[0].str.replace('ID=', '').str.replace('_4532.v6.1', '')
chlam_def = chlam_def.drop(columns = ['prediction', 'extra1', 'extra2', 'extra3'])
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '+'].index, 'strand'] = 't'
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '-'].index, 'strand'] = 'f'
chlam_def = chlam_def.loc[chlam_def['type'] == 'gene']
chlam_def['chr'] = chlam_def['chr'].str.replace('mitogenome', 'chromosome_18').str.replace('plastome', 'chromosome_19').str.replace('mating_type_plus','chromosome_20')
chlam_def = chlam_def.loc[chlam_def['chr'].str.contains('chromosome')]
chlam_def['chr'] = chlam_def['chr'].str.replace('chromosome_', '')
chlam_def[['chr', 'start', 'end']] = chlam_def[['chr', 'start', 'end']].astype(int)
chlam_def = chlam_def.sort_values(by = ['chr', 'start'])  ## sorting file according to chromosome number and gene position

#### Counting the number of genes per chromosome and per genome ####
counter1 = 0 
for chrom in chlam_def['chr'].unique():
    counter = 0
    section = chlam_def.loc[chlam_def['chr'] == chrom]
    for idx in section.index:
        counter += 1
        counter1 += 1
        chlam_def.loc[idx, 'IDg/chr'] = counter
        chlam_def.loc[idx, 'IDg/all'] = counter1
        chlam_def.loc[idx, 'IDf/all'] = counter1
        
#### Order columns in dataframe ####
columns = ['type', 'name', 'chr', 'start', 'end', 'strand', 'sens', 'IDg/chr', 'IDg/all', 'IDf/all']
chlam_def = chlam_def[columns]
chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']] = chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']].astype(int)

#### Save ####
chlam_def.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/CREN.def', sep = '\t', header = True, index = False)

In [ ]:
chlam_ch = pd.DataFrame()

for i in chlam_def['chr'].unique():
    chlam_ch.loc['chromosome', i] = 'chromosome_' + str(i)
    chlam_ch.loc['start', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()
    chlam_ch.loc['end', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()

#### Save ####
chlam_ch.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/CREN.ch', sep = '\t', header = False, index = False)

In [ ]:
from Bio import SeqIO

original_file = '/research/projects/chlamydomonas/NIexpression/genome_info/ChlreiCC4532_1/CreinhardtiiCC_4532_707_v6.1.protein.fa'
corrected_file = "/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/CREN.prt"

with open(original_file) as original, open(corrected_file, 'w') as corrected:
    records = SeqIO.parse(original_file, 'fasta')
    for record in records:
        try:
            name = record.id.split('_')[0]
            idx = chlam_def.index[chlam_def['name'] == name]
            a = chlam_def.loc[idx].drop(columns = 'type').values[0].astype(str)
            a = '\t '.join(a)
            record.id = a
            record.description = a
        except:
            continue
        SeqIO.write(record, corrected, 'fasta')

## C. incerta

In [ ]:
names = ['chr', 'prediction', 'type', 'start', 'end', 'extra1', 'sens', 'extra2', 'extra3']
chlam_def = pd.read_csv('/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_incerta/Chlamydomonas_incerta.braker2.gff3', delimiter = '\t', names = names)
chlam_def = chlam_def.loc[chlam_def['type'] == 'gene']
chlam_def = chlam_def.sort_values(by = ['chr', 'start'])
legend = { chlam_def['chr'].unique()[i]: i + 1 for i in range(len(chlam_def['chr'].unique()))}
for i in chlam_def['chr'].unique():
    chlam_def.loc[chlam_def.loc[chlam_def['chr'] == i].index, 'chr'] = legend[i]
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '+'].index, 'strand'] = 't'
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '-'].index, 'strand'] = 'f'
chlam_def['name'] = chlam_def['extra3'].str.replace('ID=', '').str.replace(';', '')
chlam_def = chlam_def.drop(columns = ['prediction', 'extra1', 'extra2', 'extra3'])

#### Counting the number of genes per chromosome and per genome ####
counter1 = 0 
for chrom in chlam_def['chr'].unique():
    counter = 0
    section = chlam_def.loc[chlam_def['chr'] == chrom]
    for idx in section.index:
        counter += 1
        counter1 += 1
        chlam_def.loc[idx, 'IDg/chr'] = counter
        chlam_def.loc[idx, 'IDg/all'] = counter1
        chlam_def.loc[idx, 'IDf/all'] = counter1
        
#### Order columns in dataframe ####
columns = ['type', 'name', 'chr', 'start', 'end', 'strand', 'sens', 'IDg/chr', 'IDg/all', 'IDf/all']
chlam_def = chlam_def[columns]
chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']] = chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']].astype(int)

#### Save ####
chlam_def.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/CINC.def', sep = '\t', header = True, index = False)

In [ ]:
chlam_ch = pd.DataFrame()

for i in chlam_def['chr'].unique():
    chlam_ch.loc['scaffold', i] = 'scaffold_' + str(i)
    chlam_ch.loc['start', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()
    chlam_ch.loc['end', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()

#### Save ####
chlam_ch.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/CINC.ch', sep = '\t', header = False, index = False)

In [ ]:
from Bio import SeqIO

original_file = '/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_incerta/Chlamydomonas_incerta.braker2.protein.fa'
corrected_file = "/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/CINC.prt"

with open(original_file) as original, open(corrected_file, 'w') as corrected:
    records = SeqIO.parse(original_file, 'fasta')
    for record in records:
        try:
            name = record.id.split('.t')[0]
            idx = chlam_def.index[chlam_def['name'] == name]
            a = chlam_def.loc[idx].drop(columns = 'type').values[0].astype(str)
            a = '\t '.join(a)
            record.id = a
            record.description = a
        except:
            continue
        SeqIO.write(record, corrected, 'fasta')

## C.schloesseri

In [ ]:
names = ['chr', 'prediction', 'type', 'start', 'end', 'extra1', 'sens', 'extra2', 'extra3']
chlam_def = pd.read_csv('/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_schloesseri/Chlamydomonas_schloesseri.braker2.gff3', delimiter = '\t', names = names)
chlam_def = chlam_def.loc[chlam_def['type'] == 'gene']
chlam_def = chlam_def.sort_values(by = ['chr', 'start']) 
legend = { chlam_def['chr'].unique()[i]: i + 1 for i in range(len(chlam_def['chr'].unique()))}
for i in chlam_def['chr'].unique():
    chlam_def.loc[chlam_def.loc[chlam_def['chr'] == i].index, 'chr'] = legend[i]
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '+'].index, 'strand'] = 't'
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '-'].index, 'strand'] = 'f'
chlam_def['name'] = chlam_def['extra3'].str.replace('ID=', '').str.replace(';', '')
chlam_def = chlam_def.drop(columns = ['prediction', 'extra1', 'extra2', 'extra3'])

#### Counting the number of genes per chromosome and per genome ####
counter1 = 0 
for chrom in chlam_def['chr'].unique():
    counter = 0
    section = chlam_def.loc[chlam_def['chr'] == chrom]
    for idx in section.index:
        counter += 1
        counter1 += 1
        chlam_def.loc[idx, 'IDg/chr'] = counter
        chlam_def.loc[idx, 'IDg/all'] = counter1
        chlam_def.loc[idx, 'IDf/all'] = counter1
        
#### Order columns in dataframe ####
columns = ['type', 'name', 'chr', 'start', 'end', 'strand', 'sens', 'IDg/chr', 'IDg/all', 'IDf/all']
chlam_def = chlam_def[columns]
chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']] = chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']].astype(int)

#### Save ####
chlam_def.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/CSCH.def', sep = '\t', header = True, index = False)

In [ ]:
chlam_ch = pd.DataFrame()

for i in chlam_def['chr'].unique():
    chlam_ch.loc['scaffold', i] = 'scaffold_' + str(i)
    chlam_ch.loc['start', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()
    chlam_ch.loc['end', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()

#### Save ####
chlam_ch.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/CSCH.ch', sep = '\t', header = False, index = False)

In [ ]:
from Bio import SeqIO

original_file = '/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_schloesseri/Chlamydomonas_schloesseri.braker2.protein.fa'
corrected_file = "/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/CSCH.prt"

with open(original_file) as original, open(corrected_file, 'w') as corrected:
    records = SeqIO.parse(original_file, 'fasta')
    for record in records:
        try:
            name = record.id.split('.t')[0]
            idx = chlam_def.index[chlam_def['name'] == name]
            a = chlam_def.loc[idx].drop(columns = 'type').values[0].astype(str)
            a = '\t '.join(a)
            record.id = a
            record.description = a
        except:
            continue
        SeqIO.write(record, corrected, 'fasta')

## JG5-1

In [ ]:
name = ['chr', 'prediction', 'type', 'start', 'end', 'extra1', 'sens', 'extra2', 'extra3']
chlam_def = pd.read_csv('/research/references/chlamydomonas/JG5.v1/JG5.v1.genome_w_organelles.gtf', delimiter = '\t', names = name, skiprows = 1)
chlam_def = chlam_def.loc[chlam_def['type'] == 'gene']
chlam_def['name'] = chlam_def['extra3'].str.split(';', expand = True)[0].str.replace('ID=', '')
chlam_def = chlam_def.drop(columns = ['prediction', 'extra1', 'extra2', 'extra3'])
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '+'].index, 'strand'] = 't'
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '-'].index, 'strand'] = 'f'
chlam_def['chr'] = chlam_def['chr'].str.replace('JG5-1_mitogenome', '91').str.replace('JG5-1_plastogenome', '92').str.replace('mating_type_plus','chromosome_20')

legend = {chlam_def['chr'].unique()[i + 2]: i+1 for i in range(90)}
for i in legend.keys():
    chlam_def.loc[chlam_def.loc[chlam_def['chr'] == i].index, 'chr'] = legend[i]
    
chlam_def[['chr', 'start', 'end']] = chlam_def[['chr', 'start', 'end']].astype(int)
chlam_def = chlam_def.sort_values(by = ['chr', 'start'])  ## sorting file according to chromosome number and gene position

#### Assigning chromosome number based on contigs ####
counter = 0
for i in chlam_def['chr'].unique():
    counter += 1
    chlam_def.loc[chlam_def.loc[chlam_def['chr'] == i].index, 'chr'] = counter

#### Counting the number of genes per chromosome and per genome ####
counter1 = 0 
for chrom in chlam_def['chr'].unique():
    counter = 0
    section = chlam_def.loc[chlam_def['chr'] == chrom]
    for idx in section.index:
        counter += 1
        counter1 += 1
        chlam_def.loc[idx, 'IDg/chr'] = counter
        chlam_def.loc[idx, 'IDg/all'] = counter1
        chlam_def.loc[idx, 'IDf/all'] = counter1
        
#### Order columns in dataframe ####
columns = ['type', 'name', 'chr', 'start', 'end', 'strand', 'sens', 'IDg/chr', 'IDg/all', 'IDf/all']
chlam_def = chlam_def[columns]
chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']] = chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']].astype(int)

### Save ####
chlam_def.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/JG05.def', sep = '\t', header = True, index = False)

In [ ]:
chlam_ch = pd.DataFrame()

for i in chlam_def['chr'].unique():
    chlam_ch.loc['chr', i] = 'scaffold_' + str(i)
    chlam_ch.loc['start', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()
    chlam_ch.loc['end', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()

#### Save ####
chlam_ch.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/JG05.ch', sep = '\t', header = False, index = False)

In [ ]:
from Bio import SeqIO

original_file = '/research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1_protein.fasta'
corrected_file = "/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/JG05.prt"

with open(original_file) as original, open(corrected_file, 'w') as corrected:
    records = SeqIO.parse(original_file, 'fasta')
    for record in records:
        try:
            name = record.id.replace('.t1', '')
            idx = chlam_def.index[chlam_def['name'] == name]
            a = chlam_def.loc[idx].drop(columns = 'type').values[0].astype(str)
            a = '\t '.join(a)
            record.id = a
            record.description = a
        except:
            try:
                name = record.id.replace('cds', 'gene')
                idx = chlam_def.index[chlam_def['name'] == name]
                a = chlam_def.loc[idx].drop(columns = 'type').values[0].astype(str)
                a = '\t '.join(a)
                record.id = a
                record.description = a
            except:
                continue 
        SeqIO.write(record, corrected, 'fasta')

## C. malaysian

In [ ]:
name = ['chr', 'prediction', 'type', 'start', 'end', 'extra1', 'sens', 'extra2', 'extra3']
chlam_def = pd.read_csv('/research/references/chlamydomonas/Cmalay.v1/Cmalay.v1.genome_w_organelles.gtf', delimiter = '\t', names = name, skiprows = 1)
chlam_def = chlam_def.loc[chlam_def['type'] == 'gene']
chlam_def['name'] = chlam_def['extra3'].str.split(';', expand = True)[0].str.replace('ID=', '')
chlam_def = chlam_def.drop(columns = ['prediction', 'extra1', 'extra2', 'extra3'])
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '+'].index, 'strand'] = 't'
chlam_def.loc[chlam_def.loc[chlam_def['sens'] == '-'].index, 'strand'] = 'f'
chlam_def['chr'] = chlam_def['chr'].str.replace('C_malay_mitogenome', '63').str.replace('C_malay_plastogenome', '64').str.replace('mating_type_plus','chromosome_20')

legend = {chlam_def['chr'].unique()[i + 2]: i+1 for i in range(62)}
for i in legend.keys():
    chlam_def.loc[chlam_def.loc[chlam_def['chr'] == i].index, 'chr'] = legend[i]
    
chlam_def[['chr', 'start', 'end']] = chlam_def[['chr', 'start', 'end']].astype(int)
chlam_def = chlam_def.sort_values(by = ['chr', 'start'])  ## sorting file according to chromosome number and gene position

#### Assigning chromosome number based on contigs ####
counter = 0
for i in chlam_def['chr'].unique():
    counter += 1
    chlam_def.loc[chlam_def.loc[chlam_def['chr'] == i].index, 'chr'] = counter

#### Counting the number of genes per chromosome and per genome ####
counter1 = 0 
for chrom in chlam_def['chr'].unique():
    counter = 0
    section = chlam_def.loc[chlam_def['chr'] == chrom]
    for idx in section.index:
        counter += 1
        counter1 += 1
        chlam_def.loc[idx, 'IDg/chr'] = counter
        chlam_def.loc[idx, 'IDg/all'] = counter1
        chlam_def.loc[idx, 'IDf/all'] = counter1
        
#### Order columns in dataframe ####
columns = ['type', 'name', 'chr', 'start', 'end', 'strand', 'sens', 'IDg/chr', 'IDg/all', 'IDf/all']
chlam_def = chlam_def[columns]
chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']] = chlam_def[['IDg/chr', 'IDg/all', 'IDf/all']].astype(int)

### Save ####
chlam_def.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/MALA.def', sep = '\t', header = True, index = False)

In [ ]:
chlam_ch = pd.DataFrame()

for i in chlam_def['chr'].unique():
    chlam_ch.loc['chr', i] = 'scaffold_' + str(i)
    chlam_ch.loc['start', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()
    chlam_ch.loc['end', i] = chlam_def.loc[chlam_def['chr'] == i]['end'].max()

#### Save ####
chlam_ch.to_csv('/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/MALA.ch', sep = '\t', header = False, index = False)

In [ ]:
from Bio import SeqIO

original_file = '/research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/C_malay_protein.fasta'
corrected_file = "/research/projects/chlamydomonas/NIexpression/data/CHROnicle/Creinhardtii/01Genomes/MALA.prt"

with open(original_file) as original, open(corrected_file, 'w') as corrected:
    records = SeqIO.parse(original_file, 'fasta')
    for record in records:
        try:
            name = record.id.replace('.t1', '')
            idx = chlam_def.index[chlam_def['name'] == name]
            a = chlam_def.loc[idx].drop(columns = 'type').values[0].astype(str)
            a = '\t '.join(a)
            record.id = a
            record.description = a
        except:
            try:
                name = record.id.replace('cds', 'gene')
                idx = chlam_def.index[chlam_def['name'] == name]
                a = chlam_def.loc[idx].drop(columns = 'type').values[0].astype(str)
                a = '\t '.join(a)
                record.id = a
                record.description = a
            except:
                continue 
        SeqIO.write(record, corrected, 'fasta')

# BLASTx - Finding Orthologous pairs

## Function - Modify GFF files

In [91]:
def modified_gff(gff):
    ''' This code is meant to remove all duplicated transcripts and leave behind the longest protein isoform'''
    mRNA = gff.loc[(gff['type'] == 'mRNA') | (gff['type'] == 'transcript')]
    mRNA['attributes'].str.split(';', expand = True)
    mRNA[['ID', 'Parent']] = mRNA['attributes'].str.split(';', expand = True)[[0, 1]]
    mRNA['genelength'] = mRNA['end'] - mRNA['start']
    
    #### Drop transcripts with same length and lowest score ####
    for parent in mRNA.loc[mRNA.duplicated(subset = ['Parent'])]['Parent'].unique():
        parent_section = mRNA.loc[mRNA['Parent'] == parent]
        max_score = parent_section['score'].max()
        mRNA.drop(parent_section.loc[parent_section['score'] != max_score].index, inplace = True)
    
    #### Remove shortest transcripts ####
    for i in mRNA['Parent'][mRNA.duplicated(subset=['Parent'])]:
        section = mRNA.loc[mRNA['Parent'] == i]
        mx = mRNA.loc[mRNA['Parent'] == i]['genelength'].max()
        mRNA.drop(section.loc[section['genelength'] != mx].index, inplace = True)
        
    mRNA.drop_duplicates(subset = ['Parent', 'genelength'], keep = 'first', inplace = True)
        
    mRNA['ID'] = mRNA['ID'].str.replace('ID=', '')
    mRNA['ID'] = mRNA['ID'].str.replace('transcript_id', '').str.replace('"', '').str.replace(' ', '')
    return mRNA

## Applying GFF function

In [182]:
name = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'] 

Cinc_gff = pd.read_csv('/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_incerta/Chlamydomonas_incerta.braker2.gff3', delimiter = '\t', names = name)
Csch_gff = pd.read_csv('/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_schloesseri/Chlamydomonas_schloesseri.braker2.gff3', delimiter = '\t', names = name)
JG5_gff = pd.read_csv('/research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.genome_w_organelles.gtf', delimiter = '\t', names = name)
Cmal_gff = pd.read_csv('/research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.genome_w_organelles.gtf', delimiter = '\t', names = name)

Cinc_gff2 = modified_gff(Cinc_gff)
Csch_gff2 = modified_gff(Csch_gff)
JG5_gff2 = modified_gff(JG5_gff)
Cmal_gff2 = modified_gff(Cmal_gff)

JG5_gff2['Parent'] = JG5_gff2['Parent'].str.replace('gene_id', '').str.replace('"', '').str.replace(' ', '')
Csch_gff2['Parent'] = Csch_gff2['Parent'].str.replace('Parent=', '').str.replace(' ', '')
Cinc_gff2['Parent'] = Cinc_gff2['Parent'].str.replace('Parent=', '').str.replace(' ', '')
Cmal_gff2['Parent'] = Cmal_gff2['Parent'].str.replace('gene_id', '').str.replace('"', '').str.replace(' ', '')

Cren_gff2 = pd.read_csv('/research/references/chlamydomonas/6.1_chlamy/annotation/CreinhardtiiCC_4532_707_v6.1.gene.gff3', delimiter = '\t', skiprows = 6, names = name)
Cren_gff2['attributes'] = Cren_gff2['attributes'].str.replace('ID=', '').str.split('_4532', expand = True)[0]
Cren_gff2 = Cren_gff2.loc[Cren_gff2['type'] == 'mRNA']
Cren_gff2['readlength'] = Cren_gff2['end'] - Cren_gff2['start']
Cren_gff2.drop_duplicates(subset = ['seqid', 'attributes', 'readlength'], keep = 'first', inplace = True)

#### GFF3 ####
for transcript in Cren_gff2['attributes'].unique():
    section = Cren_gff2.loc[Cren_gff2['attributes'] == transcript]
    transcript_max = section['readlength'].max()
    Cren_gff2.drop(section.loc[section['readlength'] != transcript_max].index, axis = 0, inplace = True)
Cren_gff2.columns = Cren_gff2.columns.str.replace('attributes', 'Parent')
Cren_gff2.sort_values(by = ['seqid', 'start'], ascending = True, inplace = True)

#### Save new GFF3 file ####
JG5_gff2.to_csv('/research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5_transcript.gff3', sep = '\t', index = False, header = True)
Csch_gff2.to_csv('/research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/Cschloesseri_mRNA.gff3', sep = '\t', index = False, header = True)
Cinc_gff2.to_csv('/research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/Cincerta_mRNA.gff3', sep = '\t', index = False, header = True)
Cmal_gff2.to_csv('/research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Malaysian_transcript.gff3', sep = '\t', index = False, header = True)
Cren_gff2.to_csv('/research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/Creinhardtii_mRNA.gff3', sep = '\t', index = False, header = True)

<ipython-input-91-8a38c5fa11a7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mRNA[['ID', 'Parent']] = mRNA['attributes'].str.split(';', expand = True)[[0, 1]]
<ipython-input-91-8a38c5fa11a7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mRNA[['ID', 'Parent']] = mRNA['attributes'].str.split(';', expand = True)[[0, 1]]
<ipython-input-91-8a38c5fa11a7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

<ipython-input-91-8a38c5fa11a7>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mRNA.drop(section.loc[section['genelength'] != mx].index, inplace = True)
<ipython-input-91-8a38c5fa11a7>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mRNA.drop_duplicates(subset = ['Parent', 'genelength'], keep = 'first', inplace = True)
<ipython-input-91-8a38c5fa11a7>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

## Getting longest transcript/protein

The code below provides a fasta file with the longest transcript per gene

In [165]:
from Bio import SeqIO

Cinc_fasta = "/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_incerta/Chlamydomonas_incerta.braker2.CDS.fa"
Csch_fasta = '/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_schloesseri/Chlamydomonas_schloesseri.braker2.CDS.fa'
Cmal_fasta = '/research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.CDS.fa'
JG5_fasta = '/research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5.CDS.fa'

Cinc_prot = '/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_incerta/Chlamydomonas_incerta.braker2.protein.fa'
Csch_prot = '/research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_schloesseri/Chlamydomonas_schloesseri.braker2.protein.fa'

############################
#### Longest Transcript ####
############################
cinc_refn = SeqIO.to_dict(SeqIO.parse(open(Cinc_fasta), 'fasta'))
o=open("/research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa", 'w')
for t_id in Cinc_gff2['ID'].unique():
    cds = "".join(cinc_refn[t_id])
    o.write(">{t_id}|Cincerta\n{cds}\n".format(t_id=t_id,cds=cds) )
o.close()

csch_refn = SeqIO.to_dict(SeqIO.parse(open(Csch_fasta), 'fasta'))
o=open("/research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_transcript.fa", 'w')
for t_id in Csch_gff2['ID'].unique():
    cds = "".join(csch_refn[t_id])
    o.write(">{t_id}|Cschloesseri\n{cds}\n".format(t_id=t_id,cds=cds) )
o.close()

cmal_refn = SeqIO.to_dict(SeqIO.parse(open(Cmal_fasta), 'fasta'))
o=open("/research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/longest_isoform_transcript.fa", 'w')
for t_id in Cmal_gff2['ID'].unique():
    try:
        cds = "".join(cmal_refn[t_id])
        o.write(">{t_id}|Cmalaysian\n{cds}\n".format(t_id=t_id,cds=cds) )
    except: continue
o.close()

jg5_refn = SeqIO.to_dict(SeqIO.parse(open(JG5_fasta), 'fasta'))
o=open("/research/projects/chlamydomonas/NIexpression/genome_info/JG5/longest_isoform_transcript.fa", 'w')
for t_id in JG5_gff2['ID'].unique():
    try:
        cds = "".join(jg5_refn[t_id])
        o.write(">{t_id}|JG5\n{cds}\n".format(t_id=t_id,cds=cds) )
    except: continue
o.close()

#################################
#### Longest Protein Isoform ####
#################################
cinc_refp = SeqIO.to_dict(SeqIO.parse(open(Cinc_prot), 'fasta'))
o=open("/research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_protein.fa", 'w')
for t_id in Cinc_gff2['ID'].unique():
    cds = "".join(cinc_refp[t_id])
    o.write(">{t_id}|Cincerta\n{cds}\n".format(t_id=t_id,cds=cds) )
o.close()

csch_refp = SeqIO.to_dict(SeqIO.parse(open(Csch_prot), 'fasta'))
o=open("/research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_protein.fa", 'w')
for t_id in Csch_gff2['ID'].unique():
    cds = "".join(csch_refp[t_id])
    o.write(">{t_id}|Cschloesseri\n{cds}\n".format(t_id=t_id,cds=cds) )
o.close()

## Running BLASTx 

In [166]:
!makeblastdb -dbtype nucl -in /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.transcript_primaryTranscriptOnly.fa
!makeblastdb -dbtype nucl -in /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa
!makeblastdb -dbtype nucl -in /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_transcript.fa

!makeblastdb -dbtype prot -in /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.proteins.longest_isoform.fasta
!makeblastdb -dbtype prot -in /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.protein_primaryTranscriptOnly.fa
!makeblastdb -dbtype prot -in /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.proteins.longest_isoform.fasta
!makeblastdb -dbtype prot -in /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_protein.fa
!makeblastdb -dbtype prot -in /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_protein.fa


# #### CREN vs JG5-1 ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.transcript_primaryTranscriptOnly.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out Cr_JG5.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/JG5/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.protein_primaryTranscriptOnly.fa -num_threads 48 -evalue 1e-10 -out JG5_Cr.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# #### CREN vs CMAL ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.transcript_primaryTranscriptOnly.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out Cr_Cmal.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.protein_primaryTranscriptOnly.fa -num_threads 48 -evalue 1e-10 -out Cmal_Cr.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# #### CREN vs CINC ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.protein_primaryTranscriptOnly.fa -num_threads 48 -evalue 1e-10 -out Cinc_Cr.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.transcript_primaryTranscriptOnly.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out Cr_Cinc.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# #### CREN vs CSCH ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.protein_primaryTranscriptOnly.fa -num_threads 48 -evalue 1e-10 -out Csch_Cr.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.transcript_primaryTranscriptOnly.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out Cr_Csch.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"

# #### JG5-1 vs CMAL ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/JG5/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out JG5_Cmal.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out Cmal_JG5.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# #### JG5-1 vs CINC ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/JG5/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out JG5_Cinc.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out Cinc_JG5.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# #### JG5-1 vs CSCH ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/JG5/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out JG5_Csch.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out Csch_JG5.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"

# #### CMAL vs CINC ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out Cinc_Cmal.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out Cmal_Cinc.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# #### CMAL vs CSCH ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out Csch_Cmal.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out Cmal_Csch.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"

# #### CINC vs CSCH ####
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out Csch_Cinc.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
# !blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out Cinc_Csch.blastx.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"



Building a new DB, current time: 05/04/2025 19:05:09
New DB name:   /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.transcript_primaryTranscriptOnly.fa
New DB title:  /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.transcript_primaryTranscriptOnly.fa
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.transcript_primaryTranscriptOnly.fa
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 17693 sequences in 1.18837 seconds.




Building a new DB, current time: 05/04/2025 19:05:10
New DB name:   /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa
New DB title:  /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa
Sequence type: Nucle

In [39]:
#### Finding gene duplicates ####
!blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/JG5/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out JG5_JG5.blastx.txt -max_hsps 1 -num_alignments 5 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
!blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out Csch_Csch.blastx.txt -max_hsps 1 -num_alignments 5 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
!blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_protein.fa -num_threads 48 -evalue 1e-10 -out Cinc_Cinc.blastx.txt -max_hsps 1 -num_alignments 5 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
!blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/longest_isoform_transcript.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.proteins.longest_isoform.fasta -num_threads 48 -evalue 1e-10 -out Cmal_Cmal.blastx.txt -max_hsps 1 -num_alignments 5 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
!blastx -query /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.transcript_primaryTranscriptOnly.fa -db /research/projects/chlamydomonas/NIexpression/genome_info/Creinhardtii_v6.1/CreinhardtiiCC_4532_707_v6.1.protein_primaryTranscriptOnly.fa -num_threads 48 -evalue 1e-10 -out Cren_Cren.blastx.txt -max_hsps 1 -num_alignments 5 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"

# Rob's Code

## BLAST line class

In [4]:
class blast_line:
    def __init__(self, l):
        l = l.strip()
        self.query = l.split()[0].split('|')[0].split('_4532')[0].split('.t')[0]
        self.subject = l.split()[1].split('|')[0].split('_4532')[0].split('.t')[0]
        self.percent_id = float(l.split()[2])
        self.alignment_length = int(l.split()[3]) 
        self.mistmatches = int(l.split()[4])
        self.gap_openings = int(l.split()[5]) 
        self.query_start = int(l.split()[6]) 
        self.query_end = int(l.split()[7]) 
        self.subject_start = int(l.split()[8]) 
        self.subject_end = int(l.split()[9])
        self.e_value = float(l.split()[10])
        self.bit_score = float(l.split()[11])
        self.qcovs = float(l.split()[12])
        self.qcovhsp = float(l.split()[13])

## Assess the similarity of pairs
--- 

 - Using the BLAST output we an look at the paired CDSs based on their 
     - qCov -Coverage - the qcoverage of the shorter CDS
     - PID - Percent Identity - the % ID of the better alignment

We can look at the relationship of the two by filtering the pairs on a series of combinations of PID and qCOV


In [ ]:
import pandas as pd

def similarity_pairs(pairs):
    df = pd.DataFrame()
    min_qcovs = [60]
    df['min_qcov'] = min_qcovs
    for min_pid in ([60]):
        col = []
        for min_qcov in min_qcovs:
            filtered_pairs = []
            for p in pairs:
                r,i = p
                r_b = Cr2Ci[r]
                i_b = Ci2Cr[i]
                pids  = (r_b.percent_id,i_b.percent_id)
                qcovs = (r_b.qcovs,i_b.qcovs)
                if max(pids) > min_pid and max(qcovs) > min_qcov:
                    #print('PASS', r, i, max(pids), max(qcovs))
                    filtered_pairs.append(p)
            col.append(len(filtered_pairs))
        df[str(min_pid)] = col
    
    return df, filtered_pairs

## Parse Reciprocal Best BLAST Hits (RBH) into pairs

In [ ]:
files = [('Cinc_Cr.blastx.txt', 'Cr_Cinc.blastx.txt', 'CINC_CREN'), ('Cmal_Cr.blastx.txt', 'Cr_Cmal.blastx.txt', 'CMAL_CREN'), 
 ('Cr_Csch.blastx.txt', 'Csch_Cr.blastx.txt', 'CREN_CSCH'), ('Cr_JG5.blastx.txt', 'JG5_Cr.blastx.txt', 'CREN_JG5-1'), 
         ('JG5_Cmal.blastx.txt', 'Cmal_JG5.blastx.txt', 'JG5-1_CMAL'), ('JG5_Cinc.blastx.txt', 'Cinc_JG5.blastx.txt', 'JG5-1_CINC'), 
 ('JG5_Csch.blastx.txt', 'Csch_JG5.blastx.txt', 'JG5-1_CSCH'), ('Cmal_Cinc.blastx.txt', 'Cinc_Cmal.blastx.txt', 'CMAL_CINC'), 
        ('Cmal_Csch.blastx.txt', 'Csch_Cmal.blastx.txt', 'CMAL_CSCH'), ('Csch_Cinc.blastx.txt', 'Cinc_Csch.blastx.txt', 'CSCH_CINC')]

for file in files:
    Cr2Ci = {}
    for l in open("/research/projects/chlamydomonas/NIexpression/data/orthologs/" + file[0]):
        b = blast_line(l)
        Cr2Ci[b.query] = b
    
    Ci2Cr = {}
    for l in open("/research/projects/chlamydomonas/NIexpression/data/orthologs/" + file[1]):
        b = blast_line(l)
        Ci2Cr[b.query] = b
    
    # Check for hits
    df_pairs = pd.DataFrame()
    pairs = []
    for r in Cr2Ci:
        hit = Cr2Ci[r]
        i = hit.subject
    #     print(r, i)
    #     print(Ci2Cr[i].subject)
        if i in Ci2Cr and Ci2Cr[i].subject == r: # Pair
            pairs.append([r,i])
#     print(len(pairs))
    
    ## Save to dataframe
    df_pairs = pd.DataFrame(pairs)
    df_pairs.to_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/' + file[2], sep = '\t', index = False, header = False)

    ## Save filtered pairs to dataframe
    filtered = pd.DataFrame(similarity_pairs(pairs)[1])
    filtered.columns = file[2].split('_')
    filtered.to_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/filteredpairs_' + file[2], sep = '\t', index = False, header = True)

## Combining RBH Orthologous Pairs

In [64]:
CREN_CINC = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CINC_CREN', delimiter = '\t', names = ['CINC', 'CREN'])
CREN_CSCH = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CREN_CSCH', delimiter = '\t', names = ['CREN', 'CSCH'])
CREN_MALA = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CMAL_CREN', delimiter = '\t', names = ['MALA', 'CREN'])
CREN_JG05 = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CREN_JG5-1', delimiter = '\t', names = ['CREN', 'JG05'])
MALA_JG05 = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/JG5-1_CMAL', delimiter = '\t', names = ['JG05', 'MALA'])
CINC_MALA = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CMAL_CINC', delimiter = '\t', names = ['MALA', 'CINC'])
CINC_CSCH = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CSCH_CINC', delimiter = '\t', names = ['CSCH', 'CINC'])
CSCH_MALA = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CMAL_CSCH', delimiter = '\t', names = ['MALA', 'CSCH'])
JG05_CINC = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/JG5-1_CINC', delimiter = '\t', names = ['JG05', 'CINC'])
JG05_CSCH = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/JG5-1_CSCH', delimiter = '\t', names = ['JG05', 'CSCH'])

result = pd.concat([CREN_CINC, CREN_CSCH, CREN_MALA, CREN_JG05, MALA_JG05, CINC_MALA, CINC_CSCH, CSCH_MALA, JG05_CINC, JG05_CSCH], axis = 0)
result.to_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/BLASTRBH_orthologous_pairs.txt', sep = '\t', index = False, header = True)

## Combining Filtered RBH Orthologous Pairs

In [2]:
directory = '/research/projects/chlamydomonas/NIexpression/data/orthologs'
filtered_orths = pd.DataFrame()
    
for filename in os.listdir(directory):
    if re.findall('filteredpairs', filename):
        f = os.path.join(directory, filename)
        data = pd.read_csv(f, delimiter = '\t')
        filtered_orths = pd.concat([filtered_orths, data], axis = 0)
species = ['CINC', 'CMAL', 'CSCH', 'JG5-1']
for i in species:
    filtered_orths[i] = filtered_orths[i].str.split('.t', expand = True)[0]
filtered_orths.to_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/filteredRBH_orthologous_pairs.txt', sep = '\t', index = False, header = True)

## Sequence Divergence of Orthologous pairs

In [78]:
directory = '/research/projects/chlamydomonas/InterspeciesCodingDivergence'
divergence = pd.DataFrame()
    
for filename in os.listdir(directory):
    if re.findall('.out.txt', filename):
        f = os.path.join(directory, filename)
        data = pd.read_csv(f, delimiter = '\t')
        data.columns = filename[:-8].split('_') + list(data.columns[2:])
        for i in data.columns[0:2]:
            try:
                data[i] = data[i].str.split('_4532', expand = True)[0]
            except: continue
            try:
                data[i] = data[i].str.split('.t', expand = True)[0]
            except: continue
        data.to_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/divergence_' + filename, sep = '\t', index = False, header = True)

# Finding Mating Type using BLASTn

In [2]:
mtplus = /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtPlus_ref.chromosome_6.fasta
mtminus = /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtMinus_ref.chromosome_6_and_mtMinus.fasta

!makeblastdb -dbtype nucl -in /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.genome_w_organelles.fasta
!makeblastdb -dbtype nucl -in /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.genome_w_organelles.fasta
!makeblastdb -dbtype nucl -in /research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_incerta/Chlamydomonas_incerta.V3_5.with_organelles.fa
!makeblastdb -dbtype nucl -in /research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_schloesseri/Chlamydomonas_schloesseri.V1_5.with_organelles.fa

#### Malaysian ####
!blastn -query /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtPlus_ref.chromosome_6.fasta -db /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.genome_w_organelles.fasta -num_threads 48 -evalue 1e-10 -out Cmal_v_mt+.blastn.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
!blastn -query /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtMinus_ref.chromosome_6_and_mtMinus.fasta -db /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.genome_w_organelles.fasta -num_threads 48 -evalue 1e-10 -out Cmal_v_mt-.blastn.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
#### JG5-1 ####
!blastn -query /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtPlus_ref.chromosome_6.fasta -db /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.genome_w_organelles.fasta -num_threads 48 -evalue 1e-10 -out mating_type/JG5_v_mt+.blastn.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
!blastn -query /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtMinus_ref.chromosome_6_and_mtMinus.fasta -db /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.genome_w_organelles.fasta -num_threads 48 -evalue 1e-10 -out mating_type/JG5_v_mt-.blastn.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
#### Chlamydomonas incerta ####
!blastn -query /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtPlus_ref.chromosome_6.fasta -db /research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_incerta/Chlamydomonas_incerta.V3_5.with_organelles.fa -num_threads 48 -evalue 1e-10 -out mating_type/Cinc_v_mt+.blastn.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
!blastn -query /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtMinus_ref.chromosome_6_and_mtMinus.fasta -db /research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_incerta/Chlamydomonas_incerta.V3_5.with_organelles.fa -num_threads 48 -evalue 1e-10 -out mating_type/Cinc_v_mt-.blastn.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
#### Chlamydomonas schloesseri ####
!blastn -query /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtPlus_ref.chromosome_6.fasta -db /research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_schloesseri/Chlamydomonas_schloesseri.V1_5.with_organelles.fa -num_threads 48 -evalue 1e-10 -out mating_type/Csch_v_mt+.blastn.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"
!blastn -query /research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/mtMinus_ref.chromosome_6_and_mtMinus.fasta -db /research/projects/chlamydomonas/Cincerta_deNovo/analysis/assembly_V3/final_assemblies_annotations/Chlamydomonas_schloesseri/Chlamydomonas_schloesseri.V1_5.with_organelles.fa -num_threads 48 -evalue 1e-10 -out mating_type/Csch_v_mt-.blastn.txt -max_hsps 1 -num_alignments 1 -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovs qcovhsp"



Building a new DB, current time: 12/23/2024 15:01:37
New DB name:   /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.genome_w_organelles.fasta
New DB title:  /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.genome_w_organelles.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/Cmalay.v1.genome_w_organelles.fasta
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 92 sequences in 1.17132 seconds.




Building a new DB, current time: 12/23/2024 15:01:39
New DB name:   /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.genome_w_organelles.fasta
New DB title:  /research/projects/chlamydomonas/NIexpression/genome_info/JG5/JG5-1.v1.genome_w_organelles.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /research/projects/chlamydomonas/NIexpression/genome_info/J

In [44]:
CREN_CINC = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CREN_CINC', delimiter = '\t', names = ['CINC', 'CREN'])
CREN_CSCH = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CREN_CSCH', delimiter = '\t', names = ['CREN', 'CSCH'])
CREN_MALA = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CREN_CMAL', delimiter = '\t', names = ['MALA', 'CREN'])
CREN_JG05 = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CREN_JG5-1', delimiter = '\t', names = ['CREN', 'JG05'])

CREN_CINC['CINC'] = CREN_CINC['CINC'].str.split('.t', expand = True)[0]
CREN_CSCH['CSCH'] = CREN_CSCH['CSCH'].str.split('.t', expand = True)[0]
CREN_MALA['MALA'] = CREN_MALA['MALA'].str.split('.t', expand = True)[0]
CREN_JG05['JG05'] = CREN_JG05['JG05'].str.split('.t', expand = True)[0]

In [79]:
a = pd.read_csv('/research/projects/chlamydomonas/NIexpression/data/raw_counts/CREN_orthologous_genes.csv', delimiter = '\t', index_col = 'Unnamed: 0')

crci = a[['CREN', 'CINC']].dropna()
crcs = a[['CREN', 'CSCH']].dropna()
crcm = a[['CREN', 'MALA']].dropna()
crjg = a[['CREN', 'JG05']].dropna()

crcs2 = pd.merge(crcs, CREN_CSCH, on = ['CREN', 'CSCH']).set_index('CREN')
crcm2 = pd.merge(crcm, CREN_MALA, on = ['CREN', 'MALA']).set_index('CREN')
crjg2 = pd.merge(crjg, CREN_JG05, on = ['CREN', 'JG05']).set_index('CREN')
crci2 = pd.merge(crci, CREN_CINC, on = ['CREN', 'CINC']).set_index('CREN')

a = crcs2.join(crcm2).join(crjg2).join(crci2).reset_index()

a.to_csv('/research/projects/chlamydomonas/NIexpression/data/orthologs/CREN_orthologous_genes.csv', sep = '\t', header = True, index = False)